# Genetic algorithm to optimize investment in the stock market

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import itertools
import matplotlib.pyplot as plt

## Assign problem hyperparameters

Two types of hyperparameters exist:
 - **Fixed parameters:** defined in the project statement
 - **Test parameters:** defined by us to test different implementations of the algorithm

In [2]:
# FIXED PARAMETERS
POPULATION_SIZE = 64 # number of individuals in population
GENERATIONS = 10000   # number of generations
EVALUATIONS = 10000   # number of evaluations
MAX_RUNS = 30 # number of runs with different random seeds

# TEST PARAMETERS
CROSSOVER_PROBABILITY = 0.8 # probability of crossover operation
MUTATION_PROBABILITY = 0.1 # probability of mutation operation
TOURNAMENT_SIZE = 5 # number of individuals participating in tournament selection

## Process datasets

Filter the data relevant for the problem in question

In [3]:
def process_data(data):
    # only relevant from 01/01/2020 onwards to 31/12/2022 (3 years)
    # convert to datetime format
    data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
    data = data[data['Date'] >= '01-01-2011']
    data = data[data['Date'] <= '31-12-2019']

    # calculate the difference between consecutive values in the 'Close' column
    value_diff = data['Close'].diff()

    # create 'Gain' and 'Loss' columns based on the 'Value_Diff'
    gain = value_diff.apply(lambda x: max(0, x))
    loss = value_diff.apply(lambda x: max(0, -x))

    # calculate the rolling sum of 'Gain' and 'Loss' for a 7-day, 14-day, and 21-day periods
    average_gain_7 =  gain.rolling(window=7).mean()
    average_loss_7 =  loss.rolling(window=7).mean()
    average_gain_14 = gain.rolling(window=14).mean()
    average_loss_14 = loss.rolling(window=14).mean()
    average_gain_21 = gain.rolling(window=21).mean()
    average_loss_21 = loss.rolling(window=21).mean()

    # calculate the Relative Strength (RS)
    rs_7 = average_gain_7/average_loss_7
    rs_14 = average_gain_14/average_loss_14
    rs_21 = average_gain_21/average_loss_21

    # calculate the Relative Strength Index (RSI)
    data['RSI7'] = 100 - (100/(1+rs_7))
    data['RSI14'] = 100 - (100/(1+rs_14))
    data['RSI21'] = 100 - (100/(1+rs_21))

    # Reset the index of the DataFrame
    data.reset_index(drop=True, inplace=True)
    return data

## Initialize DEAP

In [4]:
# create the fitness function
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# define the parameters
rsi_periods = [7, 14, 21]
lower_range = list(range(0, 101, 5))
upper_range = list(range(0, 101, 5))

# create the toolbox
toolbox = base.Toolbox()

toolbox.register("rsi_period", random.choice, rsi_periods)
toolbox.register("bound", random.choice, lower_range)

toolbox.register("attr_pool", tools.initCycle, tuple, [toolbox.rsi_period,
                                                       toolbox.bound,
                                                       toolbox.bound])

tools.initCycle
tools.initRepeat

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_pool, n=2)
toolbox.register("population",tools.initRepeat, list, toolbox.individual)

## Define simulation function

The simulation will be used in the **epigenesis** stage

In [5]:

def simulate_behaviour(individual, data):
    # extract the parameters from the individual chromosome
    (rsi_period_short, lower_short, upper_short), (rsi_period_long, lower_long, upper_long) = individual
    
    rsi_period_long = "RSI" + str(rsi_period_long)
    rsi_period_short = "RSI" + str(rsi_period_short)

    # based on the parameters, choose dates to buy and sell
    long_buys = data[(data[rsi_period_long] >= lower_long) & 
                              ((data[rsi_period_long].shift(1) < lower_long) |
                               (data[rsi_period_long].shift(1) == np.nan))].copy()
    
    short_sells = data[(data[rsi_period_short] <= upper_short) &
                                ((data[rsi_period_short].shift(1) > upper_short) |
                                 (data[rsi_period_short].shift(1) == np.nan))].copy()
    
    long_sells = []
    # check for each long buy the next long sell
    for index, row in long_buys.iterrows():
        candidate_sells = data[(data['Date'] > row['Date']) &
                               (data[rsi_period_long] >= upper_long)]
        if (candidate_sells.empty):
            new_sell = dict(data.iloc[-1])
            new_sell['index'] = len(data) - 1
        else:
            new_sell = dict(candidate_sells.iloc[0])
            new_sell['index'] = candidate_sells.index[0]
        long_sells.append(new_sell)
        drop_rows = [i for i in long_buys.index if i > index and i < new_sell['index']]
        long_buys.drop(drop_rows, inplace=True) 

    # check for each short sell the next short buy
    short_buys = []
    for index, row in short_sells.iterrows():
        candidate_buys = data[(data['Date'] > row['Date']) &
                              (data[rsi_period_short] <= lower_short)]
        if (candidate_buys.empty):
            new_buy = dict(data.iloc[-1])
            new_buy['index'] = len(data) - 1
        else:
            new_buy = dict(candidate_buys.iloc[0])
            new_buy['index'] = candidate_buys.index[0]

        short_buys.append(new_buy)
        drop_rows = [i for i in short_sells.index if i > index and i < new_buy['index']]
        short_sells.drop(drop_rows, inplace=True)


    long_sells = pd.DataFrame(long_sells)
    short_buys = pd.DataFrame(short_buys)

    # remove duplcates 
    if not long_sells.empty:
        long_sells.drop_duplicates(subset='index', inplace=True)
        long_sells.set_index('index', inplace=True)
    else:
        long_sells = pd.DataFrame(columns=data.columns)
    # remove duplicates
    if not short_buys.empty:
        short_buys.drop_duplicates(subset='index', inplace=True)
        short_buys.set_index('index', inplace=True)
    else:
        short_buys = pd.DataFrame(columns=data.columns)

    # merge buys and sells (buy long, sell long), (sell short, buy short)
    trades_long = pd.merge_asof(long_buys, long_sells, 
                                left_index=True, right_index=True, 
                                direction='forward', suffixes=('_buy', '_sell'))
    
    trades_short = pd.merge_asof(short_sells, short_buys,
                                 left_index=True, right_index=True,
                                 direction='forward', suffixes=('_sell', '_buy'))

    return (trades_long, trades_short)

# trades_long, trades_short = simulate_behaviour(individual, data)

# # plot evolution of RSI and points where buys and sells are possible
# plt.figure(figsize=(20,10))
# plt.plot(data['Date'], data['Close'], label='Close')
# plt.plot(trades_long['Date_buy'], trades_long['Close_buy'], 'ro', label='Buys Long')
# plt.plot(trades_long['Date_sell'], trades_long['Close_sell'], 'go', label='Sells Long')
# plt.plot(trades_short['Date_sell'], trades_short['Close_sell'], 'bo', label='Sells Short')
# plt.plot(trades_short['Date_buy'], trades_short['Close_buy'], 'yo', label='Buys Short')
# plt.legend(loc='upper left')
# plt.show()

toolbox.register("simulate", simulate_behaviour)

## Define objective function

The objective function will be used in the **selection** stage

In [6]:
def evaluate_results(trades_long, trades_short):
    # calculate the return on investment (ROI) for each trade
    trades_long['ROI'] = (trades_long['Close_sell'] - trades_long['Close_buy']) / trades_long['Close_buy'] * 100
    trades_short['ROI'] = (trades_short['Close_sell'] - trades_short['Close_buy']) / trades_short['Close_sell'] * 100

    # calculate the total ROI for each trade type
    average_roi_long = trades_long['ROI'].sum()
    average_roi_short = trades_short['ROI'].sum()

    # calculate the average ROI 
    average_roi = (average_roi_long + average_roi_short) / 2
    return average_roi,

toolbox.register("evaluate", evaluate_results)

## Define selection method

The selection function will be used in the **survival** stage

In [7]:
toolbox.register("select", tools.selTournament, tournsize=TOURNAMENT_SIZE)

## Define crossover method

The crossover function will be used in the **crossover** stage

In [8]:

# Crossover Hyperparameters
UNIFORM_PROBABILITY = 0.5
AVERAGE_PROBABILITY = 0.5

def crossover(individual1, individual2, method="one-point"):
    if method == "one-point":
        # 1-point crossover
        crossover_point = random.randint(0, len(individual1[0]) - 1)
        child1 = individual1[:crossover_point] + individual2[crossover_point:]
        child2 = individual2[:crossover_point] + individual1[crossover_point:]
        return child1, child2
    elif method == "two-point":
        # 2-point crossover
        crossover_point1 = random.randint(0, len(individual1[0]) - 1)
        crossover_point2 = random.randint(crossover_point1, len(individual1[0]) - 1)
        child1 = individual1[:crossover_point1] + individual2[crossover_point1:crossover_point2] + individual1[crossover_point2:]
        child2 = individual2[:crossover_point1] + individual1[crossover_point1:crossover_point2] + individual2[crossover_point2:]
        return child1, child2
    elif method == "uniform":
        # uniform crossover
        child1 = individual1.copy()
        child2 = individual2.copy()
        for i in range(len(individual1)):
            if random.random() < UNIFORM_PROBABILITY:
                child1[i], child2[i] = individual2[i], individual1[i]
        return child1, child2
    elif method == "average":
        # average crossover
        if random.random() < AVERAGE_PROBABILITY:
            child1 = (individual1+individual2)/2
        return child1, child2
    

toolbox.register("mate", crossover)

## Define mutation method

The mutation function will be used in the **mutation** stage

In [9]:
# Gaussian mutation hyperparameters
SIGMA = [0, 20, 20, 0, 20, 20]
MU = 0
GAUSSIAN_PROBABILITY = 0.7

def mutationGaussian(individual):
        (rsi_period_short, lower_short, upper_short), (rsi_period_long, lower_long, upper_long) = individual
        if random.random()<GAUSSIAN_PROBABILITY:
            lower_short = np.random.normal(MU, SIGMA[1])
            lower_short = max(0, min(100, lower_short))
            lower_short = lower_short - (lower_short % 5)
        if random.random()<GAUSSIAN_PROBABILITY:
            upper_short = np.random.normal(MU, SIGMA[2])
            upper_short = max(0, min(100, upper_short))
            upper_short = upper_short - (upper_short % 5)
        if random.random()<GAUSSIAN_PROBABILITY:
            lower_long = np.random.normal(MU, SIGMA[4])
            lower_long = max(0, min(100, lower_long))
            lower_long = lower_long - (lower_long % 5)
        if random.random()<GAUSSIAN_PROBABILITY:
            upper_long = np.random.normal(MU, SIGMA[5])
            upper_long = max(0, min(100, upper_long))
            upper_long = upper_long - (upper_long % 5)

        individual = (rsi_period_short, lower_short, upper_short), (rsi_period_long, lower_long, upper_long)
        return individual,
    
# Single mutation hyperparameters
MUTATION_BOUND = 10 # mutation bound - hyperparameter
MUTATION_WINDOW = 7 # windows bound mutation - hyperparameter
MUTATION_SINGLE_PROBABILITY = 0.5 # probability of single mutation - hyperparameter

def mutationSingle(individual):
        # mutation direction is either -1 (decrease) or 1 (increase)
        (rsi_period_short, lower_short, upper_short), (rsi_period_long, lower_long, upper_long) = individual
        mutation_direction = random.choice([-1, 1])
        # mutate the parameter, consideration of bounds
        if random.random() < MUTATION_SINGLE_PROBABILITY:
            mutation_direction = random.choice([-1, 1])
            rsi_period_short = max(7, min(21, rsi_period_short + mutation_direction * MUTATION_BOUND))
        elif random.random() < MUTATION_SINGLE_PROBABILITY:
            mutation_direction = random.choice([-1, 1])
            lower_short = max(0, min(100, lower_short + mutation_direction * MUTATION_BOUND))
        elif random.random() < MUTATION_SINGLE_PROBABILITY:
            mutation_direction = random.choice([-1, 1])
            upper_short = max(0, min(100, upper_short + mutation_direction * MUTATION_BOUND))
        elif random.random() < MUTATION_SINGLE_PROBABILITY:
            mutation_direction = random.choice([-1, 1])
            rsi_period_long = max(7, min(21, rsi_period_long + mutation_direction *  MUTATION_BOUND))
        elif random.random() < MUTATION_SINGLE_PROBABILITY:
            mutation_direction = random.choice([-1, 1])
            lower_long = max(0, min(100, lower_long + mutation_direction * MUTATION_BOUND))
        elif random.random() < MUTATION_SINGLE_PROBABILITY:
            mutation_direction = random.choice([-1, 1])
            upper_long = max(0, min(100, upper_long + mutation_direction * MUTATION_BOUND))
        individual = (rsi_period_short, lower_short, upper_short), (rsi_period_long, lower_long, upper_long)
        return individual,

def mutationInversion(individual):
    (rsi_period_short, lower_short, upper_short), (rsi_period_long, lower_long, upper_long) = individual
    if random.random() < MUTATION_SINGLE_PROBABILITY:
        rsi_period_short = 7 + 21 - rsi_period_short
    if random.random() < MUTATION_SINGLE_PROBABILITY:
        rsi_period_long = 7 + 21 - rsi_period_long
    if random.random() < MUTATION_SINGLE_PROBABILITY:
        lower_short = 100 - lower_short
    if random.random() < MUTATION_SINGLE_PROBABILITY:
        upper_short = 100 - upper_short
    if random.random() < MUTATION_SINGLE_PROBABILITY:
        lower_long = 100 - lower_long
    if random.random() < MUTATION_SINGLE_PROBABILITY:
        upper_long = 100 - upper_long
    individual = (rsi_period_short, lower_short, upper_short), (rsi_period_long, lower_long, upper_long)
    return individual,

def mutate(individual, method="single"):
    if method == "single":
        mutant = mutationSingle(individual) # single mutation
    elif method == "gaussian":
        mutant = mutationGaussian(individual)
    elif method == "inversion":
        mutant = mutationInversion(individual)
        return mutant,

toolbox.register("mutate", mutate)

# toolbox.register("mate", tools.cxTwoPoint)
# toolbox.register("mutate", tools.mutUniformInt, low=0, up=100, indpb=0.1)

## Normalized Box Plot

In [10]:

def normalize(data):
    
    pass

def boxplot(data):
    normalized_data = normalize(data)
    for i in range(len(normalized_data)):
        plt.boxplot(normalized_data[i])
        plt.show()
    pass


## Histograms ##

## Run the algorithm

In [11]:
def algorithm(data):
    list_of_output = []
    convergence = 3 # maximum number of generations without improvement
    std_threshold = 1e-4 # threshold for standard deviation
    
    for i in range(MAX_RUNS):
        count = 0 # counter for convergence
        random.seed(i) # set random seed for each run
        print('--Run: {0}--\n'.format(i))
        evaluations = 0
        # create the initial population
        pop = toolbox.population(n=POPULATION_SIZE)
        print(pop)
        
        print('Starting evolution...')

        # evaluate the entire population
        behaviours = list(map(lambda ind: toolbox.simulate(ind, data), pop))
        fitnesses = list(map(lambda bhv: toolbox.evaluate(bhv[0], bhv[1]), behaviours)) 
        evaluations += len(pop)
        
        for ind, fit in zip(pop, fitnesses):
            ind.fitness.values = fit

        print('Evaluated {0} individuals'.format(len(pop)))

        for g in range(GENERATIONS):
            print('--Generation: {0}--'.format(g))
            # select the next generation individuals
            offspring = toolbox.select(pop, len(pop))
            # clone the selected individuals
            offspring = list(map(toolbox.clone, offspring))
            # apply crossover and mutation on the offspring
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < CROSSOVER_PROBABILITY:
                    toolbox.mate(child1, child2, 'two-point')
                    del child1.fitness.values
                    del child2.fitness.values
            for mutant in offspring:
                if random.random() < MUTATION_PROBABILITY:
                    toolbox.mutate(mutant, 'inversion')
                    del mutant.fitness.values

            # evaluate the individuals with an invalid fitness
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            behaviours = map(lambda ind: toolbox.simulate(ind, data), invalid_ind)
            fitnesses = map(lambda bhv: toolbox.evaluate(bhv[0], bhv[1]), behaviours)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
            evaluations += len(invalid_ind)
            print('Evaluated {0} individuals'.format(len(invalid_ind)))

            pop[:] = offspring

            fits = [ind.fitness.values[0] for ind in pop]

            length = len(pop)
            mean = sum(fits) / length
            sum2 = sum(x*x for x in fits)
            std = abs(sum2 / length - mean**2)**0.5

            print('Min: {0}'.format(min(fits)))
            print('Max: {0}'.format(max(fits)))
            print('Avg: {0}'.format(mean))
            print('Std: {0}'.format(std))
            print('Current evaluations : ', evaluations)
            
            dictionary = {
                'DATASET': df_name,
                'RUN': i,
                'MIN': min(fits),
                'MAX': max(fits),
                'AVG': mean,
                'STD': std,
                'best_individual': pop[np.argmax(fits)]
            }
        
            if (std<std_threshold):
                count +=1
            
            if (count == convergence or evaluations > EVALUATIONS):
                list_of_output.append(dictionary)
                print('-- End of (successful) evolution --')
                break
    df = pd.DataFrame(list_of_output)       
    df.to_csv('results.csv', mode='a', header=False, index=False)    

    # obtain best individuals in each run for test scheme
    best_individuals = df['best_individual'].tolist()
    # save to csv
    df = pd.DataFrame(best_individuals)
    df.to_csv('best_individuals.csv', mode='a', header=False, index=False)

    # obtain averages of the runs and minimums, maximums, averages, and standard deviations of the best individuals
    min_values = [item['MIN'] for item in list_of_output]
    max_values = [item['MAX'] for item in list_of_output]
    avg_values = [item['AVG'] for item in list_of_output]
    std_values = [item['STD'] for item in list_of_output]
    min_average = sum(min_values) / len(min_values)
    max_average = sum(max_values) / len(max_values)
    avg_average = sum(avg_values) / len(avg_values)
    std_average = sum(std_values) / len(std_values)


    min_from_output = min(list_of_output, key=lambda x:x['MIN'])
    max_from_output = max(list_of_output, key=lambda x:x['MAX'])
    avg_from_output = max(list_of_output, key=lambda x:x['AVG'])
    std_from_output = min(list_of_output, key=lambda x:x['STD'])
    dictionary_output = {
        'DATASET': df_name,
        'MIN': min_from_output['MIN'],
        'MAX': max_from_output['MAX'],
        'AVG': avg_from_output['AVG'],
        'STD': std_from_output['STD'],
        'AVG_MIN': min_average,
        'AVG_MAX': max_average,
        'AVG_AVG': avg_average,
        'AVG_STD': std_average,
    }
    # save results for table in report
    df2 = pd.DataFrame(dictionary_output, index=[0])
    df2.to_csv('results_for_table.csv', mode='a', header=False, index=False)

            
folder_path = 'data'

file_list = os.listdir(folder_path)

data_dictionary = {}
RSI_dictionary = {}
price_dictionary = {}
date_dictionary = {}

for file_name in file_list:
    if file_name.endswith('.csv'):
        # extract the base name of the file (without the extension)
        df_name = os.path.splitext(file_name)[0]
        # construct the full file path
        file_path = os.path.join(folder_path, file_name)
        # read the CSV file into a DataFrame with the base name as the variable name
        data_dictionary[df_name] = pd.read_csv(file_path, sep=';', usecols=['Date', 'Close'])
        data = process_data(data_dictionary[df_name])
        data_dictionary[df_name] = data

for df_name in data_dictionary:
    algorithm(data_dictionary[df_name])



--Run: 0--

[[(14, 65, 5), (14, 80, 75)], [(14, 45, 75), (14, 90, 30)], [(21, 20, 45), (7, 15, 95)], [(14, 85, 95), (7, 45, 15)], [(21, 10, 50), (14, 85, 15)], [(14, 65, 50), (21, 100, 30)], [(21, 75, 70), (21, 40, 5)], [(21, 0, 10), (21, 60, 100)], [(7, 95, 75), (14, 35, 50)], [(21, 10, 30), (21, 35, 35)], [(7, 85, 70), (7, 10, 50)], [(21, 75, 15), (14, 85, 45)], [(21, 15, 85), (14, 85, 30)], [(21, 85, 90), (14, 70, 10)], [(21, 60, 50), (21, 35, 45)], [(7, 30, 25), (7, 95, 40)], [(14, 10, 10), (21, 20, 20)], [(7, 10, 85), (21, 60, 80)], [(14, 80, 35), (7, 90, 65)], [(21, 40, 70), (14, 100, 55)], [(7, 50, 95), (7, 75, 90)], [(21, 50, 30), (7, 0, 40)], [(7, 35, 55), (7, 50, 65)], [(7, 15, 20), (21, 35, 5)], [(21, 100, 85), (21, 10, 0)], [(7, 100, 30), (21, 90, 15)], [(14, 10, 55), (7, 5, 95)], [(7, 30, 25), (21, 15, 75)], [(7, 5, 0), (21, 65, 95)], [(7, 40, 10), (7, 10, 100)], [(14, 55, 65), (7, 5, 80)], [(14, 5, 95), (7, 60, 30)], [(14, 55, 75), (21, 25, 30)], [(7, 25, 25), (14, 80, 40

KeyboardInterrupt: 